# Simplified architectures

This script takes the domain clusters specified in 'custom_domain_clusters.csv' and uses this to translate the myraids of domain accessions from uniprot to non-redundant domain architectures for all phage lytic proteins in PhaLP. It outputs these to the file 'simplified_architectures.csv', which is used in further analyses.

In [1]:
import csv
from itertools import combinations, permutations, chain
import matplotlib.pyplot as plt
import mysql.connector
import numpy as np
import pandas as pd
import pickle

user = 'xxxx' # fill in MySQL user
password = 'xxxx' # fill in MySQL pw
db = 'xxxx' # fill in MySQL database name
socket = '/x/x/x/x.sock' # fill in MySQL unix socket

## Wrangle data

### remove known contaminants from further investigation

In [2]:
### These have been established to be contimants and hence not actual phage lytic proteins
### their non-redundant architectures are thus not generated
skip = ['G9IA41']

### Accessions

In [3]:
### query database
cnx = mysql.connector.connect(user=user, password=password, database=db, unix_socket=socket)
cursor = cnx.cursor()
cursor.execute("SELECT DISTINCT(UniProt_ID) FROM UniProt;")

acc = []
for i in cursor:
    if i[0] not in skip: #bypass contaminated sequences
        acc.append(i[0])
print("Amount of unique uniprot accessions: ", len(acc))

Amount of unique uniprot accessions:  11837


### Domains

In [4]:
acc2dom = {}
for i in acc:
    acc2dom[i] = []
    cnx = mysql.connector.connect(user=user, password=password, database=db, unix_socket=socket)
    cursor = cnx.cursor()
    query = "SELECT l.domains_ID, l.start, l.end FROM link_UniRef_domains as l JOIN UniProt as up WHERE up.UniProt_ID = '" + i + "' AND up.UniRef_ID = l.UniRef_ID;"
    cursor.execute(query)

    for j in cursor:
        if j[0]:
            acc2dom[i].append([j[0], j[1], j[2]])

### Domain clusters

In [5]:
dom2clst = {}
sorted_clsts = []
cbds = []
r = 0
with open('../domain-clustering/custom_domain_clusters.csv') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        if r != 0:
            dom2clst[row[0]] = row[2]
            if row[2] not in sorted_clsts:
                sorted_clsts.append(row[2])
                if row[3] == 'CBD':
                    cbds.append(row[2])
        r += 1
print(len(dom2clst.keys()), 'domains were simplified into', len(sorted_clsts), 'clusters.')

188 domains were simplified into 105 clusters.


## Simplification functions

In [6]:
def check_overlap(doms):
    ### order domains on upper boundary
    ordered_doms = sorted(doms, key=lambda x: x[2])
    
    if ordered_doms[0][1] >= ordered_doms[1][1]:
        return 'Full overlap'
    elif ordered_doms[0][2] > ordered_doms[1][1]:
        return 'Partial overlap'
    elif ordered_doms[0][2] <= ordered_doms[1][1]:
        return 'No overlap'

def get_smallest(doms):
    len1 = doms[0][2] - doms[0][1]
    len2 = doms[1][2] - doms[1][1]
    if len1 < len2:
        return doms[0]
    return doms[1]

def simplify(accession):
    all_doms = acc2dom[accession]
    ### remove domains that are too specific, i.e. aren't clustered
    valid_arch = [[dom2clst[i[0]], i[1], i[2]] for i in all_doms if i[0] in dom2clst.keys()]
    combos = combinations(valid_arch, 2)
    for i in combos: ### examine domains pairwise
        if check_overlap(i) != 'No overlap':
            if i[0][0] == i[1][0]: ### if overlapping domains belong to the same cluster, remove the smallest
                if get_smallest(i) in valid_arch:
                    valid_arch.remove(get_smallest(i))
            else:
                acc_of_interest.append(accession)
    return sorted(valid_arch, key=lambda x: x[2]) ### sort remaining domains

def first_repeated_cbd(architecture, cbds):
    if len(architecture) == len(set(architecture)):
        return None
    
    for idx in range(len(architecture)-1):
        if architecture[idx] == architecture[idx+1]:
            if architecture[idx] in cbds:
                return idx
    return None
    
def condense_repeats(architecture, cbds, show_before=False):
    tmp = architecture[:]
    idx = first_repeated_cbd(tmp, cbds)
    while idx != None:
        del tmp[idx]
        idx = first_repeated_cbd(tmp, cbds)
    if show_before:
        return architecture, tmp
    else:
        return tmp


## Simplify architectures

### As is

In [7]:
acc2arch = {}
acc_of_interest = []
for i in acc:
    acc2arch[i] = simplify(i)
acc_of_interest = set(acc_of_interest)
print('There are', len(acc_of_interest), 'peculiar architectures that should be checked.\n')
print('There are', len([i for i in acc2arch.keys() if acc2arch[i] == []]), 'PhaLPs without viable architectures,')

to_rem = [i for i in acc2arch.keys() if acc2arch[i] == []]
for i in sorted(to_rem, reverse=True):
    acc2arch.pop(i)
    
print(len(acc2arch), 'simplified PhaLP-architectures remain.')

There are 154 peculiar architectures that should be checked.

There are 253 PhaLPs without viable architectures,
11584 simplified PhaLP-architectures remain.


In [8]:
with open('simplified_architectures.csv', 'w') as f:
    for acc in acc2arch.keys():
        row = str(acc)
        for arch in acc2arch[acc]:
            row += ',' + arch[0] #+ ' '
            ### uncomment the following line to output domain boundaries as well:
            #row += '(' + str(arch[1]) + '-' + str(arch[2]) + ')' 
        row += '\n'
        f.write(row)
    f.close()

### Condense repeated CBDs

In [10]:
acc2arch_condensed = {}
for k, v in acc2arch.items():
    architecture = [x[0] for x in v]
    acc2arch_condensed[k] = condense_repeats(architecture, cbds)
    
with open('simplified_architectures_condensed.csv', 'w') as f:
    for acc in acc2arch_condensed.keys():
        row = str(acc)
        for dom in acc2arch_condensed[acc]:
            row += ',' + dom
        row += '\n'
        f.write(row)
    f.close()

## Abundances

In [15]:
### get individual domain accession abundances
sorted_doms = list(dom2clst.keys())
dom_ab = np.zeros((len(sorted_doms)), dtype=int)
for i in acc2dom:
    for j in acc2dom[i]:
        if j[0] in sorted_doms:
            dom_ab[sorted_doms.index(j[0])] += 1
print(dom_ab)

[ 186  186  628  757    8  276  422  425  425  415  640  235   18   34
  462  179  414    1    1   21   17  250 2078  302 1923 2291  269  290
  294  314   54  955  958  747  260   19  103    2   55    1  210  248
  632  724  782   38    1   10    4  541  504   30   90    1   83    4
    3  388  304    1   32   10   10  255  250  743  283    1  162    6
    6    1  466  181  424  121 1546  642   24   24    1   16    1   21
   17 1138  517   52   33   21  101   94   79  273  203  529  329   32
    2    1   17    9    1    1    1    1    5    6   95    2    4    3
  405  328    1  104  100  264  259    2  258    2    7    1    4   10
    2    2    1    1    1    1    1    1    2   10    4    7    1    2
    1    7    8    4    1    2   52    3    3    2    1    1    1    3
    7    3    1    2    1    2    3    3    3    1    1    1    1    2
    1    1    4    3    1    1    2    3    1    5    2    5    7    1
    2    1   33   15    1    1]


In [18]:
### get domain cluster abundances
clst_ab = np.zeros((len(sorted_clsts)), dtype=int)
for i in acc2arch:
    for j in acc2arch[i]:
        clst_ab[sorted_clsts.index(j[0])] += 1
print(clst_ab)

[ 186  757    8  276  427  640  235   18  552    2   22  250 2309  294
  314   54 1127  747  260   19 1081  782   38   10  541   30  388   10
  254  748    1  162    6    1 1834   24 2202   32    2    1   17    1
    1    5    6   95    4    3  405  328    1  269    2  258    2    7
    1    4   10    2    1    1   11   10    1    2   51    5    1    1
    1    3    7    3    3    1    2    3    3    3    1    1    1    1
    2    1    1    4    3    1    1    2    3    1    5    2    5    7
    1    2    1   33   15    1    1]
